In [1]:
import logging
import time

import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from joblib import parallel_backend, Memory

from time_series_models.estimators import XgbRegressor, IdentityRegressor
from time_series_models.filters import RowFilter, InterpolateLinear
from time_series_models.processes import AmiHourlyForecast, PVForecast
from time_series_models.time_series_models import RegularTimeSeriesModel
from time_series_models.time_series_models import make_domain
from time_series_models.data_fetchers.ami_fetcher import AmiFetcher
from time_series_models.data_fetchers.hrrr_fetcher import HrrrFetcher

Memory("/tmp/time_series_models_cache").clear()

logger = logging.getLogger()
logging.Formatter.converter = time.gmtime
logging.basicConfig(
    format="%(asctime)s.%(msecs)03dZ %(processName)s %(threadName)s %(levelname)s:%(name)s:%(message)s",
    datefmt="%Y-%m-%dT%H:%M:%S",
    level=logging.INFO,
)
logger.info("configured logging")

pd.set_option('display.max_columns', None)
sklearn.set_config(display="diagram")

AmiFetcher.GCS_BUCKET = "seto2243-forecasting"
# HrrrFetcher.HRRR_GCS_BUCKET = ""

2023-05-30 20:55:36.690897: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-30 20:55:36.742078: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-30 20:55:36.743211: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 20:55:37.670198: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/venv/lib/python3.10/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
class Model(
    AmiHourlyForecast,
    RowFilter,
    InterpolateLinear,
    XgbRegressor,
    RegularTimeSeriesModel,
):
    pass

config = dict(
    lags = np.array([24, 48, 168], dtype="timedelta64[h]"),
    day_of_week=True,
    harmonics = np.array([24, 168, 365*24], dtype="timedelta64[h]")
)

instance = Model(**config)
instance.model

Pipeline(steps=[('feature_builder',
                 ColumnTransformer(transformers=[('count_locs',
                                                  FunctionTransformer(func=<function count_domain_locs_into_column at 0x7fe0c7afbbe0>),
                                                  [0]),
                                                 ('correlated_load',
                                                  Pipeline(steps=[('extend',
                                                                   FunctionTransformer(func=<function shift_time at 0x7fe0c7afb7f0>,
                                                                                       kw_args={'extend': True,
                                                                                                'time_delta': numpy.timedelta64(168,'h'),
                                                                                                'tstep': nump...
                              importance_type='gain',
                              interaction_constraints=None, learning_rate=None,
                              max_delta_step=None, max_depth=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=100,
                              n_jobs=None, num_parallel_tree=None,
                              random_state=None, reg_alpha=None,
                              reg_lambda=None, scale_pos_weight=None,
                              subsample=None, tree_method=None,
                              validate_parameters=None, verbosity=None))])

In [3]:
instance.fit("2021-01-15", "2021-06-01", "55797646")

In [4]:
instance.predict_dataframe(
    "2021-03-15",
    "2021-03-15T04",
    "55797646",
    range=True,
)


predicted   true
location date_time                            
55797646 2021-03-15 00:00:00   1.906638  0.640
         2021-03-15 01:00:00   1.630097  0.472
         2021-03-15 02:00:00   1.227366  0.640
         2021-03-15 03:00:00   1.051105  0.872
         2021-03-15 04:00:00   1.478648    NaN

In [5]:
pv_config = dict(
    site_config_mapping="RESOURCE_SELF",
    site_meter_mapping=None,
    site_latlong_mapping="RESOURCE_SELF",
    source_mode="12_hour_horizon",
    lags=None,
)



class PVForecastModel(
    PVForecast,
    IdentityRegressor,
    RegularTimeSeriesModel,
):
    pass

pv_instance = PVForecastModel(**pv_config)


pv_instance.model


Pipeline(steps=[('feature_builder',
                 ColumnTransformer(transformers=[('count_locs',
                                                  FunctionTransformer(func=<function count_domain_locs_into_column at 0x7fe0c7afbbe0>),
                                                  [0]),
                                                 ('pv_physical',
                                                  Pipeline(steps=[('hrrr_domain',
                                                                   ColumnTypeTransformer(transformers=[('domain',
                                                                                                        'passthrough',
                                                                                                        [0]),
                                                                                                       ('hrrr_data',
                                                                                                        HrrrFetcher(location_mapper=<function PVForecast.__init__.<...
                                                                   FunctionTransformer(func=<function nan_like at 0x7fe0c7b08940>))]),
                                                  [0])])),
                ('data_monitor_unfiltered', ForecastDataMonitor(use_locs=True)),
                ('feature_filter',
                 Pipeline(steps=[('passthrough', 'passthrough')])),
                ('data_monitor_filtered', ForecastDataMonitor()),
                ('drop_location_count',
                 FunctionTransformer(func=<function drop_first_column at 0x7fe0c7b08550>)),
                ('estimator', IdentityEstimator())])